In [1]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 뼈대!

In [2]:
train_acc_id = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity_704.csv').acc_id
test_acc_id = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_activity_704.csv').acc_id

In [3]:
#뼈대 만들기
train_acc_id = train_acc_id.to_frame()
test_acc_id = test_acc_id.to_frame()

In [4]:
train_acc_id.index = train_acc_id['acc_id']
test_acc_id.index = test_acc_id['acc_id']

In [5]:
train_acc_id = train_acc_id.drop(['acc_id'], axis = 1)
test_acc_id = test_acc_id.drop(['acc_id'], axis = 1)

## 길드!

In [6]:
train_acc_id['guild_cnt'] = 0
test_acc_id['guild_cnt'] = 0

In [7]:
# 길드 dataset 로딩
train_guild_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_guild.csv')
test_guild_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_guild.csv')

In [8]:
train_guild = train_guild_load
test_guild = test_guild_load

In [9]:
for gid in train_guild.guild_member_acc_id:
    
    for value in gid.split(','):
        try:
            train_acc_id.loc[value, 'guild_cnt'] += 1
        except:
            pass

In [10]:
for gid in test_guild.guild_member_acc_id:
    
    for value in gid.split(','):
        try:
            test_acc_id.loc[value, 'guild_cnt'] += 1
        except:
            pass

## 지불!

In [11]:
def select_2min_value(train_column, test_column):
    train = list(set(train_column))
    test = list(set(test_column))
    
    train.sort()
    test.sort()
    
    result = []
    result.extend(train[0:2])
    result.extend(test[0:2])
    
    result = list(set(result))
    result.sort()
    return result[0:2]

In [12]:
# 길드 dataset 로딩
train_payment_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_payment.csv').sort_values(by='acc_id', ascending=True)
test_payment_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_payment.csv').sort_values(by='acc_id', ascending=True)

In [13]:
train_payment = train_payment_load
test_payment = test_payment_load

In [14]:
two_min = select_2min_value(train_payment.payment_amount, test_payment.payment_amount)

In [15]:
train_payment.payment_amount = np.around((train_payment.payment_amount - two_min[0])/(two_min[1] - two_min[0]) * 33)
test_payment.payment_amount = np.around((test_payment.payment_amount - two_min[0])/(two_min[1] - two_min[0]) * 33)

In [16]:
train_temp = train_payment[train_payment['payment_week'] == 1].drop(['payment_week', 'payment_amount'], axis=1)
test_temp = test_payment[test_payment['payment_week'] == 1].drop(['payment_week', 'payment_amount'], axis=1)

In [17]:
for i in range(1,9):
    train_temp['payment_amount_' + str(i)] = train_payment[train_payment['payment_week'] == i].payment_amount.values
    test_temp['payment_amount_' + str(i)] = test_payment[test_payment['payment_week'] == i].payment_amount.values

In [18]:
train_result = pd.merge(train_acc_id, train_temp, how = 'inner', on = 'acc_id')
test_result = pd.merge(test_acc_id, test_temp, how = 'inner', on = 'acc_id')

In [24]:
train_result['is_payment'] = np.sum(train_result.loc[:,'payment_amount_1':], axis=1) > 0
test_result['is_payment'] = np.sum(test_result.loc[:,'payment_amount_1':], axis=1) > 0

In [27]:
train_result.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_guild_payment.csv')
test_result.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_guild_payment.csv')